In [9]:
#update local run search procedure
  
import numpy as np
import copy
from VRPTW_base import vrptw_graph
from threading import Event


class Ant:
    def__init__(self,graph:vrptw_graph,start_index=0):
        super()
        self.graph = graph
        self.current_index = start_index
        self.vehicle_load = 0
        self.vehicle_travel_time = 0
        self.travel_path = [start_index]
        self.arrival_time = [0]

        self.index_to_visit = list(range(graph.node_num))
        self.index_to_visit.remove(start_index)

        self.total_travel_distance = 0
    
    def clear(self):
        self.travel_path.clear()
        self.index_to_visit.clear()
        
    def move_to_next_index(self, next_index):
        #update ant path
        self.travel_path.append(next_index)
        self.total_travel_distance += self.graph.node_dist_mat[self.current_index][next_index]
        
        dist = self.graph.node_dist_mat[self.current_index][next_index]
        self.arrival_time.append(self.vehicle_travel_time+dist)
        
        if self.graph.nodes[next_index].is_depot:
            #if next location is a hospital, you need to clear the ambulance load, and reset the vehicle time
            self.vehicle_load = 0
            self.vehicle_travel_time = 0
            
        else:
            #update ambulance load, travel distance, time
            self.vehicle_load += self.graph.nodes[next_index].demand
            #if the arrival time is earlier than the time window required by the patient(ready_time) you need to wait
            
            self.vehicle_travel_time+=dist + max(self.graph.nodes[next_index].ready_time - self.vehicle_travel_time-dist,0)+
            self.graph.nodes[next_index].service_time
            self.index_to_visit.remove(next_index)
            
        self.current_index = next_index
        
    def index_to_visit_empty(self):
        return len(self.index_to_visit)==0
    
    def get_active_vehicles_num(self):
        return self.travel_path.count(0)-1
    
    def check_condition(self, next_index) -> bool:
        #checking whether moving to the next point satifies the condition
        
        if self.vehicle_load+self.graph.nodes[next_index].demand>self.graph.vehicle_capacity:
            return False
        
        dist = self.graph.node_dist_mat[self.current_index][next_index]
        wait_time = max(self.graph.nodes[next_index].ready_time-self.vehicle_travel_time-dist,0)
        service_time = self.graph.nodes[next_index].service_time
        
        #check whether you can go back to the hospital after visiting a patient
        
        if self.vehicle_travel_time + dist + wait_time + service_time + self.graph.node_dist_mat[next_index][0] > self.graph.nodes[0].due_time:
            return False
        #cannot serve patients out of due time
        if self.vehicle_travel_time + dist > self.graph.nodes[next_index].due_time:
            return False
        
        return True
    
    def cal_next_index_meet_constraints(self):
        #find all patients reachable from the current location(ant.current_index)
        next_index_meet_constraints= []
        for next_ind in self.index_to_visit:
            if self.check_condition(next_ind):
                next_index_meet_constraints.append(next_ind)
                
            return next_index_meet_constraints
        def cal_nearest_next_index(self,next_index_list):
            #select from the patients to be selected, the patients closest to the current location
            current_ind = self.current_index
            nearest_ind = next_index_list[0]
            min_dist = self.graph.node_dist_mat[current_ind][next_index_list[0]]
            
            for next_ind in next_index_list[1:]:
                dist = self.graph.node_dist_mat[current_ind][next_ind]
                if dist<min_dist:
                    min_dist = dist
                    nearest_ind = next_ind 
                    
            return nearest_ind
        
    @static method 
    def cal_total_travel_distance(graph: vrptw_graph, travel_path):
        distance = 0 
        current_ind = travel_path[0]
        for next_ind in travel_path[1:]:
            distance += graph.node_dist_mat[current_ind][next_ind]
            current_ind = next_ind
        return distance
    

    def try_insert_on_path(self, node_id, stop_event: Event):
        #inserting node_id into current travel_path and the inserted position cannot violate the load,time, and travel distance 
        #restrictions. if there are multiple locations, find the best one
        
        best_insert_index = None
        best_distance = None
        
        for insert_index in range(len(self.travel_path)):
            if stop_event.is_set():
            # print('[try_insert_on_path]: receive stop event')
                return
            if self.graph.nodes[self.travel_path[insert_index]].is_depot:
                continue
                
            #find the nearest depot in front of nearest_index 
            front_depot_index = insert_index
            while front_depot_index>= and not self.graph.nodes[self.travel_path[front_depot_index]].is_depot:
                front_depot -=1
            front_depot_index = max(front_depot_index,0)
            #check ant depart from front depot index
            
            check_ant = Ant(self.graph, self.travel_path[front_depot_index])
            #let check_ant go through the point where the subscript in the path starts from front_depot_index to index-1
            for i in range(front_depot_index+1, insert_index):
                check_ant.move_to_next_index(self.travel_path[i])
                
            #starting to visit nodes in the sorted index_to_visit
            if check_ant.check_condition(node_id):
                check_ant.move_to_next_index(node_id)
            else:
                continue
            #if the node_id can be reached, ensure that the vehicle can travel back to the depot
            for next_ind in self.travel_path[insert_index:]:
                if stop_event.is_set():
                    # print('[try_insert_on_path]: receive stop event')
                    return
                if check_ant.check_condition(next_ind):
                    check_ant.move_to_next_index(next_ind)
                    
                #if you return to the depot'
                if self.graph.nodes[next_index].is_depot:
                    temp_front_index = self.travel_path[insert_index-1]
                    temp_back_index = self.travel_path[insert_index]
                    check_ant_distance = self.total_travel_distance - self.graph.node_dist_mat[temp_front_index][temp_back_index]
                    +self.graph.node_dist_mat[temp_front_index][node_id]+self.graph.node_dist_mat[node_id][temp_back_index]
                    
                if best_distance is None or check_ant_distance<best_distance:
                    best_distance = check_ant_distance
                    best_insert_index = insert_index
                    
                break
                
                #if it is not possible to return to the previous depot, return to the previous level
                else:
                    break
                    
            return best_insert_index
                    
                
    

ModuleNotFoundError: No module named 'VRPTW_base'